In [1]:
import os.path
import pickle
import re
import json
import tensorflow as tf
import pandas as pd
import os
import numpy as np
from datetime import datetime
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.layers import Activation
sns.set()

In [2]:
#os.chdir(r"C:\Users\aiah2\Downloads")
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

%cd gdrive/MyDrive/Bus_Tracker

import os
for file in os.listdir():
  print(file)

Mounted at /content/gdrive/
/content/gdrive/MyDrive/Bus_Tracker
finalData
BusTracker_Models.ipynb


In [3]:
with open('finalData', 'rb') as f:
    data = pickle.load(f)

In [4]:
data.shape

(649629, 19)

In [5]:
data.columns

Index(['direction', 'StopSequence', 'DayInWeek_friday', 'DayInWeek_monday',
       'DayInWeek_saturday', 'DayInWeek_sunday', 'DayInWeek_thursday',
       'DayInWeek_tuesday', 'DayInWeek_wednesday', 'preD1', 'preD2',
       'timeCategory_Unknown', 'timeCategory_d1', 'timeCategory_d2',
       'timeCategory_d3', 'timeCategory_d4', 'timeCategory_d5',
       'timeCategory_d6', 'targetTime'],
      dtype='object')

In [6]:
data

,direction,StopSequence,DayInWeek_friday,DayInWeek_monday,DayInWeek_saturday,DayInWeek_sunday,DayInWeek_thursday,DayInWeek_tuesday,DayInWeek_wednesday,preD1,preD2,timeCategory_Unknown,timeCategory_d1,timeCategory_d2,timeCategory_d3,timeCategory_d4,timeCategory_d5,timeCategory_d6,targetTime
2,1,3,0,0,0,1,0,0,0,34935,34870,0,0,0,1,0,0,0,34986
3,1,4,0,0,0,1,0,0,0,35014,34935,0,0,0,1,0,0,0,35065
4,1,5,0,0,0,1,0,0,0,35099,35014,0,0,0,1,0,0,0,35142
5,1,6,0,0,0,1,0,0,0,35169,35099,0,0,0,1,0,0,0,35195
6,1,7,0,0,0,1,0,0,0,35246,35169,0,0,0,1,0,0,0,35308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141262,0,7,0,0,1,0,0,0,0,66651,66554,0,0,0,0,0,0,1,66693
1141264,0,9,0,0,1,0,0,0,0,66800,66758,0,0,0,0,0,0,1,66833
1141275,0,20,0,0,1,0,0,0,0,67318,67282,0,0,0,0,0,0,1,67337
1141276,0,21,0,0,1,0,0,0,0,67359,67318,0,0,0,0,0,0,1,67384


In [7]:
# Split features and target variable
column_to_move = 'targetTime'
new_df = data.targetTime
data.drop(column_to_move, axis=1, inplace = True)
data = pd.concat([data, new_df], axis=1)

In [8]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [9]:
# Split the data into training and testing sets
X_train_, X_test_, y_train_, y_test_ = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_)
X_test = scaler.transform(X_test_)

In [ ]:
# Save X_train
with open('X_train.pkl', 'wb') as f:
    pickle.dump(X_train, f)

# Save X_test
with open('X_test.pkl', 'wb') as f:
    pickle.dump(X_test, f)

# Save y_train
with open('y_train.pkl', 'wb') as f:
    pickle.dump(y_train, f)

# Save y_test
with open('y_test.pkl', 'wb') as f:
    pickle.dump(y_test, f)

In [11]:
# Build the ANN model
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model.fit(X_train, y_train_, epochs=100, batch_size=32)

# Evaluate the model
loss = model.evaluate(X_test, y_test_)
print("Mean Squared Error:", loss)

# Make predictions
predictions = model.predict(X_test)

# Calculate the absolute difference between true and predicted arrival times
diff = np.abs(y_test_ - predictions.flatten())

within_range = np.logical_and(diff >= -60, diff <= 60)

# Calculate the accuracy within the range
accuracy = np.sum(within_range) / len(y_test_)

print("Accuracy within -60 to +60 seconds range: {:.2%}".format(accuracy))

Epoch 1/100
16241/16241 [==============================] - 49s 3ms/step - loss: 1581572736.0000 - mae: 35892.2188
Epoch 2/100
16241/16241 [==============================] - 46s 3ms/step - loss: 81176040.0000 - mae: 5544.5269
Epoch 3/100
16241/16241 [==============================] - 47s 3ms/step - loss: 5615869.5000 - mae: 1521.7466
Epoch 4/100
16241/16241 [==============================] - 45s 3ms/step - loss: 3034996.0000 - mae: 1138.4677
Epoch 5/100
16241/16241 [==============================] - 43s 3ms/step - loss: 2181121.7500 - mae: 958.7209
Epoch 6/100
16241/16241 [==============================] - 43s 3ms/step - loss: 1470939.2500 - mae: 778.4218
Epoch 7/100
16241/16241 [==============================] - 43s 3ms/step - loss: 912463.7500 - mae: 598.8044
Epoch 8/100
16241/16241 [==============================] - 42s 3ms/step - loss: 524654.5625 - mae: 444.6664
Epoch 9/100
16241/16241 [==============================] - 41s 3ms/step - loss: 286412.5625 - mae: 332.8267
Epoch 10/100


Result:

TEST MAE =  63.973

Accuracy within -60 to +60 seconds range: 65.09%

In [ ]:
# Build the ANN model
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model.fit(X_train, y_train_, epochs=100, batch_size=32)

# Evaluate the model
loss = model.evaluate(X_test, y_test_)
print("Mean Squared Error:", loss)

# Make predictions
predictions = model.predict(X_test)

# Calculate the absolute difference between true and predicted arrival times
diff = np.abs(y_test_ - predictions.flatten())

within_range = np.logical_and(diff >= -60, diff <= 60)

# Calculate the accuracy within the range
accuracy = np.sum(within_range) / len(y_test_)

print("Accuracy within -60 to +60 seconds range: {:.2%}".format(accuracy))

Epoch 1/100
16241/16241 [==============================] - 46s 3ms/step - loss: 151080208.0000 - mae: 4605.5376
Epoch 2/100
16241/16241 [==============================] - 43s 3ms/step - loss: 290102.7812 - mae: 313.2631
Epoch 3/100
16241/16241 [==============================] - 45s 3ms/step - loss: 51446.4375 - mae: 115.2622
Epoch 4/100
16241/16241 [==============================] - 43s 3ms/step - loss: 27128.7539 - mae: 98.3545
Epoch 5/100
16241/16241 [==============================] - 45s 3ms/step - loss: 15336.7100 - mae: 81.4971
Epoch 6/100
16241/16241 [==============================] - 43s 3ms/step - loss: 12972.5430 - mae: 74.0813
Epoch 7/100
16241/16241 [==============================] - 45s 3ms/step - loss: 12452.2510 - mae: 71.9732
Epoch 8/100
16241/16241 [==============================] - 44s 3ms/step - loss: 12047.4893 - mae: 70.4541
Epoch 9/100
16241/16241 [==============================] - 43s 3ms/step - loss: 11720.2627 - mae: 69.0003
Epoch 10/100
16241/16241 [===========

Result:

TEST MAE =  49.7287

Accuracy within -60 to +60 seconds range: 76.49%

>>>>>> Saving the best trained model in TASK#:


In [ ]:
# After training, save the model to a file
model.save('simpleANN_BusTracker_2_64_32.h5')

In [14]:
# Build the MLP model
model = Sequential()

model.add(Dense(units=128, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=32, activation='relu', input_dim=X_train.shape[1]))

# Add the output layer
model.add(Dense(units=1, activation='relu'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
model.fit(X_train, y_train_, epochs=100, batch_size=32)

# Evaluate the model on the test data
loss, mae = model.evaluate(X_test, y_test_)
print("Mean Squared Error:", loss)
print(f"Test MAE: {mae}")

# Make predictions
predictions = model.predict(X_test)

# Calculate the absolute difference between true and predicted arrival times
diff = np.abs(y_test_ - predictions.flatten())

within_range = np.logical_and(diff >= -60, diff <= 60)

# Calculate the accuracy within the range
accuracy = np.sum(within_range) / len(y_test_)

print("Accuracy within -60 to +60 seconds range: {:.2%}".format(accuracy))

Epoch 1/100
16241/16241 [==============================] - 54s 3ms/step - loss: 27054370.0000 - mae: 897.0555
Epoch 2/100
16241/16241 [==============================] - 51s 3ms/step - loss: 26094.4648 - mae: 116.0736
Epoch 3/100
16241/16241 [==============================] - 53s 3ms/step - loss: 23951.6562 - mae: 111.0575
Epoch 4/100
16241/16241 [==============================] - 51s 3ms/step - loss: 22478.6348 - mae: 107.2282
Epoch 5/100
16241/16241 [==============================] - 51s 3ms/step - loss: 21834.1230 - mae: 105.4293
Epoch 6/100
16241/16241 [==============================] - 51s 3ms/step - loss: 20774.5820 - mae: 102.4683
Epoch 7/100
16241/16241 [==============================] - 51s 3ms/step - loss: 20370.3418 - mae: 100.8615
Epoch 8/100
16241/16241 [==============================] - 51s 3ms/step - loss: 19728.0273 - mae: 98.9627
Epoch 9/100
16241/16241 [==============================] - 51s 3ms/step - loss: 19736.9336 - mae: 98.9252
Epoch 10/100
16241/16241 [==========

Result:

TEST MAE =  60.31

Accuracy within -60 to +60 seconds range: 62.57%

In [ ]:
# Build the ANN model
model = Sequential()

model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=32, activation='relu', input_dim=X_train.shape[1]))

# Add the output layer
model.add(Dense(units=1, activation='relu'))

# Compile the model with additional metrics
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train_, epochs=100, batch_size=32)

# Evaluate the model on the test data
loss, mae = model.evaluate(X_test, y_test_)
print("Mean Squared Error:", loss)

# Make predictions
predictions = model.predict(X_test)

# Calculate the absolute difference between true and predicted arrival times
diff = np.abs(y_test - predictions.flatten())

within_range = np.logical_and(diff >= -60, diff <= 60)

# Calculate the accuracy within the range
accuracy = np.sum(within_range) / len(y_test_)

print("Accuracy within -60 to +60 seconds range: {:.2%}".format(accuracy))

Epoch 1/100
16241/16241 [==============================] - 43s 3ms/step - loss: 179075744.0000 - mae: 5358.0127
Epoch 2/100
16241/16241 [==============================] - 42s 3ms/step - loss: 630705.5000 - mae: 495.8897
Epoch 3/100
16241/16241 [==============================] - 43s 3ms/step - loss: 41743.3594 - mae: 132.2942
Epoch 4/100
16241/16241 [==============================] - 43s 3ms/step - loss: 17862.1602 - mae: 87.8977
Epoch 5/100
16241/16241 [==============================] - 43s 3ms/step - loss: 13832.3672 - mae: 76.3667
Epoch 6/100
16241/16241 [==============================] - 43s 3ms/step - loss: 12549.4180 - mae: 72.0089
Epoch 7/100
16241/16241 [==============================] - 42s 3ms/step - loss: 11921.2900 - mae: 69.6099
Epoch 8/100
16241/16241 [==============================] - 43s 3ms/step - loss: 11557.7139 - mae: 68.2197
Epoch 9/100
16241/16241 [==============================] - 45s 3ms/step - loss: 11302.5361 - mae: 67.2175
Epoch 10/100
16241/16241 [===========

Result:

TEST MAE = 57.2

Accuracy within -60 to +60 seconds range: 68.87%